In [1]:
import os
import sys
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

2023-12-07 00:12:18.585937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 00:12:18.585965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 00:12:18.586661: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 00:12:18.591566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 00:12:19.138231: W tensorflow/compiler/tf2

Using TensorFlow backend


In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print(tf.__version__)

GPU found
2.15.0


2023-12-07 00:12:20.065366: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:12:20.082299: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:12:20.082491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-07 00:12:20 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-07 00:12:20 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-07 00:12:20 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-07 00:12:20 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [4]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)
print(y_test.shape)
print(x_test.shape)

In [5]:
# Model creation
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes,
            weight_vae=0.25, weight_clf=20)

2023-12-07 00:12:20 [INFO]: Classificator available for the latent space of the autoencoder
2023-12-07 00:12:20.396993: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:12:20.397218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:12:20.397352: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more 

In [6]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=20, batch_size=128)

trained_model = pipe(train_data=train_data)

2023-12-07 00:12:20 [INFO]: +++ training_pipeline +++
2023-12-07 00:12:20 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-07_00-12-20


Epoch 1/20


2023-12-07 00:12:34.761982: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904



Epoch 1: total_loss improved from inf to 489305.15625, saving model to ../output_dir/training_pipeline_2023-12-07_00-12-20/model.weights.h5
469/469 - 22s - 46ms/step - clf_loss: 6.9770 - kl_loss: 1949444.3750 - reconstruction_loss: 7217.4473 - total_loss: 489305.1562
Epoch 2/20

Epoch 2: total_loss improved from 489305.15625 to 2271.93921, saving model to ../output_dir/training_pipeline_2023-12-07_00-12-20/model.weights.h5
469/469 - 4s - 8ms/step - clf_loss: 2.2408 - kl_loss: 1691.2861 - reconstruction_loss: 7217.2075 - total_loss: 2271.9392
Epoch 3/20

Epoch 3: total_loss improved from 2271.93921 to 2143.78833, saving model to ../output_dir/training_pipeline_2023-12-07_00-12-20/model.weights.h5
469/469 - 4s - 8ms/step - clf_loss: 2.1835 - kl_loss: 1183.4087 - reconstruction_loss: 7217.0698 - total_loss: 2143.7883
Epoch 4/20

Epoch 4: total_loss improved from 2143.78833 to 2076.71436, saving model to ../output_dir/training_pipeline_2023-12-07_00-12-20/model.weights.h5
469/469 - 3s - 7

In [7]:
model.load_weights(filepath='../output_dir/training_pipeline_2023-11-25_00-50-30/model.weights.h5')

In [8]:
def plot_label_clusters(vae,  must be equal, but ar
    # Display a 2D plot of the digit classes in the latent space
    _, _, z = vae.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z[:, 0], z[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()
    if include_flow:
      z = vae.flow.inverse(z)
      plt.figure(figsize=(12, 10))
      plt.scatter(z[:, 0], z[:, 1], c=labels)
      plt.colorbar()
      plt.xlabel("z[0]")
      plt.ylabel("z[1]")
      plt.show()

SyntaxError: invalid syntax (87367603.py, line 1)

In [9]:
plot_label_clusters(model, train_data, y_train)
plot_label_clusters(model, test_data, y_test)

NameError: name 'plot_label_clusters' is not defined

In [10]:
from sklearn.metrics import classification_report

#y_train_predict = model.predict(train_data)
y_test_predict =  model.predict(test_data)

print(np.argmax(y_test_predict['clf'], axis=1))
print(np.argmax(y_test, axis=1))

target_names = ['number 0', 'number 1', 'number 2', 'number 3', 'number 4', 'number 5', 'number 6', 'number 7', 'number 8', 'number 9']

#classification_report(y_train, np.argmax(y_train_predict['clf'], axis=1), target_names=target_names)
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_predict['clf'], axis=1), target_names=target_names))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
              precision    recall  f1-score   support

    number 0       0.98      0.98      0.98       980
    number 1       0.96      0.99      0.97      1135
    number 2       0.98      0.97      0.97      1032
    number 3       0.97      0.98      0.97      1010
    number 4       0.98      0.96      0.97       982
    number 5       0.98      0.95      0.97       892
    number 6       0.97      0.99      0.98       958
    number 7       0.96      0.97      0.97      1028
    number 8       0.97      0.95      0.96       974
    number 9       0.97      0.97      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



In [11]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
test set results: [
	 accuracy_score: 0.97 
]


0.97